In [ ]:
# ============== Imports ==============
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

# ============== Constantes ==============
OLLAMA_API = "http://localhost:11434/api/chat"  # Assicurati che Ollama sia in esecuzione
HEADERS = {"Content-Type": "application/json"}
MODEL = "gemma3:1b"

# ============== Definizione del Prompt di Sistema ==============
system_prompt = (
    "Sei un assistente che analizza il contenuto di un sito web e fornisce un riassunto dettagliato. "
    "Devi ignorare tutto il testo che riguarda la navigazione del sito, come menù, barra di ricerca, "
    "collegamenti esterni, elementi di navigazione (come header, footer, e sidebar), form, annunci, "
    "immagini, video, e altre sezioni che non sono parte del contenuto principale o rilevante. "
    "Il riassunto che fornirai dovrà essere conciso ma informativo, includendo solo le informazioni principali, "
    "come descrizioni di prodotti, servizi offerti, notizie o articoli, ma evitando ogni tipo di dettaglio superfluo. "
    "Rispondi utilizzando il formato Markdown, organizzando il testo in modo chiaro e leggibile. "
    "Se il sito contiene elenchi, evidenzia i punti principali in formato elenco puntato. "
    "Assicurati che il riassunto sia strutturato in modo tale da facilitare la comprensione veloce del contenuto."
)

# ============== Funzione per generare il prompt dell'utente ==============
def user_prompt_for(website):
    user_prompt = f"Stai visualizzando un sito web intitolato **{website.title}**.\n"
    user_prompt += "Il contenuto di questo sito è il seguente; fornisci un breve riassunto in formato Markdown.\n"
    user_prompt += "Se ci sono prodotti, notizie o annunci, includili nel riassunto.\n\n"
    user_prompt += website.text
    return user_prompt

# ============== Classe Website ==============
class Website:
    """Classe per rappresentare un sito web analizzato con BeautifulSoup."""
    def __init__(self, url: str, timeout: int = 20):
        self.url = url
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Titolo
        self.title = soup.title.string.strip() if soup.title and soup.title.string else "Senza titolo"
        
        # Estrazione del corpo del sito
        body = soup.body or soup
        for tag in body.select("script, style, noscript, svg, header, footer, nav, form, aside, img, input"):
            tag.decompose()  # Rimuove gli elementi non necessari

        # Estrazione del testo
        raw_text = body.get_text(separator="\n", strip=True)
        self.text = "\n".join([line.strip() for line in raw_text.splitlines() if len(line.split()) >= 2])  # Rimuove le righe vuote

# ============== Funzione per inviare il messaggio a Ollama ==============
def summarize(url):
    website = Website(url)  # Crea un oggetto Website per il sito web
    messages = [{"role": "system", "content": system_prompt}]  # Invia il prompt di sistema
    user_message = user_prompt_for(website)  # Genera il messaggio dell'utente
    messages.append({"role": "user", "content": user_message})  # Aggiungi il messaggio dell'utente

    # Invia la richiesta a Ollama
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']  # Restituisce il contenuto del riassunto generato

# ============== Funzione per visualizzare il riassunto in formato Markdown ==============
def display_summary(url):
    summary = summarize(url)  # Chiama la funzione summarize per ottenere il riassunto
    display(Markdown(summary))  # Visualizza il riassunto in Markdown

# ============== Esecuzione ==============
display_summary("https://play-sense.shop/")  # Inserisci l'URL del sito di cui vuoi il riassunto
